# 第7章：数据规整化：清理、转换、合并、重塑

# 主要内容—— 3.字符串处理  

3.1 字符串对象方法  

3.2 正则表达式  

3.3 pandas中的字符串向量化函数
 

# 3.1字符串对象方法  
1. 分隔和连接字符串：split()  strip()  join()  
2. 锁定字符串位置相关：关键字in，index()  find()  count()


大部分字符串的处理，使用python内建函数就足够了，比如split用于分隔逗号分隔的字符串

In [1]:
val = 'a,b, guido'

In [2]:
val.split()

['a,b,', 'guido']

strip用于分隔空格分隔的字符串

In [3]:
pieces = [x.strip() for x in val.split(',')]
pieces

['a', 'b', 'guido']

In [4]:
first, second, third = pieces

In [5]:
first + '::' + second + '::' + third

'a::b::guido'

但这种方法并不python，更快的方法是直接用join方法：

In [6]:
'::'.join(pieces)

'a::b::guido'

关键字 in 可以用来检测某字符串是否包含另一字符串

In [7]:
'guido' in val

True

In [8]:
val

'a,b, guido'

In [9]:
val.index(',')

1

In [12]:
val.find(',')

1

如果要找的string不存在的话，index会报错。而find会返回-1

In [11]:
val.index(':')

ValueError: substring not found

In [14]:
val.find(':')

-1

count会返回一个substring出现的次数

replace会取代一种出现方式（pattern）。也通常用于删除pattern，传入一个空字符串即可

In [15]:
val.replace(',', '::')

'a::b:: guido'

In [16]:
val.replace(',', '')

'ab guido'

In [19]:
from IPython.display import Image
Image(url= "2.png",width=500, height=300)

# 3.2 正则表达式(Regular Expressions)

# 1.正则表达式简介 

Regular Expression，在代码中常简写为regex、regexp或RE），计算机科学的一个概念。正则表达式使用单个字符串来描述、匹配一系列匹配某个句法规则的字符串。在很多文本编辑器里，正则表达式通常被用来检索、替换那些匹配某个模式的文本。

正则表达式并不是Python的一部分。正则表达式是用于处理字符串的强大工具，拥有自己独特的语法以及一个独立的处理引擎，效率上可能不如str自带的方法，但功能十分强大。

# 2.正则表达式基础

** （一）入门**

** 判断一个字符串是否是合法的Email的方法是：**


1. 创建一个匹配Email的正则表达式；

2. 用该正则表达式去匹配用户的输入来判断是否合法。

**因为正则表达式也是用字符串表示的，所以，我们要首先了解如何用字符来描述字符**

在正则表达式中，如果直接给出字符，就是精确匹配。用\d可以匹配一个数字，\w可以匹配一个字母或数字，所以：

- '00\d'可以匹配'007'，但无法匹配'00A'；

- '\d\d\d'可以匹配'010'；

- '\w\w\d'可以匹配'py3'；

.可以匹配任意字符，所以：

- 'py.'可以匹配'pyc'、'pyo'、'py!'等等。

要匹配变长的字符，在正则表达式中，用\*表示任意个字符（包括0个），用+表示至少一个字符，用?表示0个或1个字符，用{n}表示n个字符，用{n,m}表示n-m个字符：

来看一个复杂的例子：\d{3}\s+\d{3,8}。

我们来从左到右解读一下：

- \d{3}表示匹配3个数字，例如'010'；

- \s可以匹配一个空格（也包括Tab等空白符），所以\s+表示至少有一个空格，例如匹配' '，' '等；

- \d{3,8}表示3-8个数字，例如'1234567'。

综合起来，上面的正则表达式可以匹配以任意个空格隔开的带区号的电话号码。

如果要匹配'010-12345'这样的号码呢？由于'-'是特殊字符，在正则表达式中，要用'\'转义，所以，上面的正则是\d{3}\-\d{3,8}。

但是，仍然无法匹配'010 - 12345'，因为带有空格。所以我们需要更复杂的匹配方式。

** （二）进阶**


要做更精确地匹配，可以用[]表示范围，比如：

- [0-9a-zA-Z\_]可以匹配一个数字、字母或者下划线；

- [0-9a-zA-Z\_]+可以匹配至少由一个数字、字母或者下划线组成的字符串，比如'a100'，'0_Z'，'Py3000'等等；

- \[a-zA-Z\_][0-9a-zA-Z\_]\*可以匹配由字母或下划线开头，后接任意个由一个数字、字母或者下划线组成的字符串，也就是Python合法的变量；

- [a-zA-Z\_][0-9a-zA-Z\_]{0, 19}更精确地限制了变量的长度是1-20个字符（前面1个字符+后面最多19个字符）。

A|B可以匹配A或B，所以(P|p)ython可以匹配'Python'或者'python'。

^表示行的开头，^\d表示必须以数字开头。

\$表示行的结束，\d\$表示必须以数字结束。

py也可以匹配'python'，但是加上^py\$就变成了整行匹配，就只能匹配'py'了。

# 3.re模块  

re有以下三个类别：  
1. patther matching（模式匹配）
2. substitution（替换）
3. splitting（分割）。

Python提供re模块，包含所有正则表达式的功能。由于Python的字符串本身也用\转义，所以要特别注意：

In [36]:
s1 = 'ABC\\-001' # Python的字符串
# 对应的正则表达式字符串变成：
# 'ABC\-001'

因此我们强烈建议使用Python的r前缀，就不用考虑转义的问题了：

In [34]:
s2 = r'ABC\-001' # Python的字符串
# 对应的正则表达式字符串不变：
# 'ABC\-001'

例子

In [44]:
import re
re.match(r'^\d{3}\-\d{3,8}$', '010-12345')

<_sre.SRE_Match object; span=(0, 9), match='010-12345'>

In [45]:
re.match(r'^\d{3}\-\d{3,8}$', '010 12345')

match()方法判断是否匹配，如果匹配成功，返回一个Match对象，否则返回None。常见的判断方法就是：

In [47]:
test = '用户输入的字符串'
if re.match(r'正则表达式', test):
    print('ok')
else:
    print('failed')

failed


# **分组**

除了简单地判断是否匹配之外，正则表达式还有提取子串的强大功能。用()表示的就是要提取的分组（Group）。比如：

^(\d{3})-(\d{3,8})$分别定义了两个组，可以直接从匹配的字符串中提取出区号和本地号码

In [48]:
m = re.match(r'^(\d{3})-(\d{3,8})$', '010-12345')

In [49]:
m

<_sre.SRE_Match object; span=(0, 9), match='010-12345'>

In [50]:
m.group(0)

'010-12345'

In [51]:
m.group(1)

'010'

In [52]:
m.group(2)

'12345'

如果正则表达式中定义了组，就可以在Match对象上用group()方法提取出子串来

In [54]:
m.groups()

('010', '12345')

# ** 贪婪匹配**  
 
所谓贪婪匹配，就是匹配尽可能多的字符。比如：匹配出数字后面的0

In [55]:
re.match(r'^(\d+)(0*)$', '102300').groups()

('102300', '')

由于\d+采用贪婪匹配，直接把后面的0全部匹配了，结果0\*只能匹配空字符串了。  

必须让\d+采用非贪婪匹配（也就是尽可能少匹配），才能把后面的0匹配出来，加个?就可以让\d+采用非贪婪匹配

In [56]:
re.match(r'^(\d+?)(0*)$', '102300').groups()

('1023', '00')

# ** 编译 **  

当我们在Python中使用正则表达式时，re模块内部会干两件事情：

1. 编译正则表达式，如果正则表达式的字符串本身不合法，会报错；

2. 用编译后的正则表达式去匹配字符串。


如果一个正则表达式要重复使用几千次，出于效率的考虑，我们可以预编译该正则表达式

In [57]:
text = "foo    bar\t baz  \tqux"

In [58]:
regex = re.compile('\s+')

In [59]:
regex.split(text)

['foo', 'bar', 'baz', 'qux']

使用re.compile创建一个regex object是被强烈推荐的，如果你打算把一个表达式用于很多string上的话，这样可以节省CPU的资源。

# match、search和findall

如果想要得到符合regex的所有结果，以一个list结果返回，可以使用findall方法

In [25]:
regex.findall(text)

['    ', '\t ', '  \t']

match和search，与findall关系紧密。不过findall会返回所有匹配的结果，而search只会返回第一次匹配的结果。更严格地说，match只匹配string开始的部分。这里举个例子说明，我们想要找到所有的邮件地址：

In [60]:
#需要匹配的字符串
text = """Dave dave@google.com 
          Steve steve@gmail.com 
          Rob rob@gmail.com 
          Ryan ryan@yahoo.com """
# 正则表达式
pattern = r'[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}'

In [61]:
regex = re.compile(pattern, flags=re.IGNORECASE)

其中，re.IGNORECASE使匹配对大小写不敏感

使用findall找到一组邮件地址：

In [62]:
regex.findall(text)

['dave@google.com', 'steve@gmail.com', 'rob@gmail.com', 'ryan@yahoo.com']

search返回text中的第一个匹配结果。match object能告诉我们找到的结果在text中开始和结束的位置

In [63]:
m = regex.search(text)

In [64]:
m

<_sre.SRE_Match object; span=(5, 20), match='dave@google.com'>

In [65]:
text[m.start():m.end()]

'dave@google.com'

regex.match返回None，因为它只会在pattern存在于stirng开头的情况下才会返回匹配结果：

In [66]:
print(regex.match(text))

None


而sub返回一个新的string，把pattern出现的地方替换为我们指定的string：

In [68]:
print(regex.sub('REDACTED', text))

Dave REDACTED 
          Steve REDACTED 
          Rob REDACTED 
          Ryan REDACTED 


假设你想要找到邮件地址，同时，想要把邮件地址分为三个部分，username, domain name, and domain suffix.（用户名，域名，域名后缀）。需要给每一个pattern加一个括号：

In [69]:
pattern = r'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})'

In [70]:
regex = re.compile(pattern, flags=re.IGNORECASE)

In [71]:
m = regex.match('wesm@bright.net')

In [72]:
m.groups()

('wesm', 'bright', 'net')

In [73]:
regex.findall(text)

[('dave', 'google', 'com'),
 ('steve', 'gmail', 'com'),
 ('rob', 'gmail', 'com'),
 ('ryan', 'yahoo', 'com')]

# 3.3 pandas中的字符串向量化函数

一些复杂的数据清理中，string会有缺失值：

In [74]:
import pandas as pd
import numpy as np

In [77]:
data1 = {'Dave': 'dave@google.com', 'Steve': 'steve@gmail.com', 
        'Rob': 'rob@gmail.com', 'Wes': np.nan}

In [78]:
data = pd.Series(data1)
data

Dave     dave@google.com
Rob        rob@gmail.com
Steve    steve@gmail.com
Wes                  NaN
dtype: object

In [79]:
data.isnull()

Dave     False
Rob      False
Steve    False
Wes       True
dtype: bool

可以把一些字符串方法和正则表达式（用lambda或其他函数）用于每一个value上，通过data.map，但是这样会得到NA(null)值。为了解决这个问题，series有一些数组导向的方法可以用于字符串操作，来跳过NA值。这些方法可以通过series的str属性；比如，我们想检查每个电子邮箱地址是否有'gmail' with str.contains:

In [81]:
data.str

In [82]:
data.str.contains('gmail')

Dave     False
Rob       True
Steve     True
Wes        NaN
dtype: object

正则表达式也可以用，配合任意的re选项，比如IGNORECASE:

In [83]:
pattern

'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\\.([A-Z]{2,4})'

In [84]:
data.str.findall(pattern, flags=re.IGNORECASE)

Dave     [(dave, google, com)]
Rob        [(rob, gmail, com)]
Steve    [(steve, gmail, com)]
Wes                        NaN
dtype: object

有很多方法用于向量化。比如str.get或index索引到str属性：

In [85]:
matches = data.str.match(pattern, flags=re.IGNORECASE)
matches

C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: FutureWarning: In future versions of pandas, match will change to always return a bool indexer.
  if __name__ == '__main__':


Dave     (dave, google, com)
Rob        (rob, gmail, com)
Steve    (steve, gmail, com)
Wes                      NaN
dtype: object

为了访问嵌套list里的元素，我们可以传入一个index给函数

In [86]:
matches.str.get(1)

Dave     google
Rob       gmail
Steve     gmail
Wes         NaN
dtype: object

也可以使用这个语法进行切片

In [87]:
data.str[:5]

Dave     dave@
Rob      rob@g
Steve    steve
Wes        NaN
dtype: object

一些字符串向量化的方法有：

In [89]:
Image(url= "3.png",width=600, height=400)